# Test Evaluation Library

In [1]:
import os
import sys
import json
import collections
from tqdm import tqdm

import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
from torch.nn import DataParallel
import torchvision
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

sys.path.append("..")

from data import StyledCoco
import models
from CONFIG import CONFIG
import data
import data.custom_transforms as custom_transforms
import data.data_processing as data_processing
import lib.arguments as arguments
import lib.utils as utils
import lib.metrics as metrics
import lib.pose_parsing as pose_parsing 
import lib.visualizations as visualizations

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
test_exp = "test/experiment_2020-06-04_08-06-38"
test_exp = os.path.join(CONFIG["paths"]["experiments_path"], test_exp)

In [4]:
exp_data = utils.load_experiment_parameters(test_exp)
exp_data["dataset"]["dataset_name"] = "coco"

## Setting up Dataset and Model

In [5]:
_, valid_loader = data.load_dataset(exp_data=exp_data, train=False,
                                    validation=True, shuffle_train=True,
                                    shuffle_valid=False)

loading annotations into memory...
Done (t=1.41s)
creating index...
index created!


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.OpenPose()
pretrained_path = os.path.join(CONFIG["paths"]["pretrained_path"], "OpenPose", "checkpoint_iter_370000.pth")
model.load_pretrained(pretrained_path)

model = DataParallel(model).to(device)
model.eval()
pass

## Inference and Computing Results

In [7]:
with torch.no_grad():
    utils.reset_predictions_file(test_exp)
    for i, (img, _, _, _, metadata) in enumerate(valid_loader):
        if(i % 5 == 0):
            print(f"Processing image {i}")
        if(i == 15):
            break
        img = img.float().to(device)
        stages_output = model(img)
        heatmaps = stages_output[-2].cpu().detach().numpy()[0,:]
        pafs = stages_output[-1].cpu().detach().numpy()[0,:]

        # computing predictions, loading previous results and appending new ones
        cur_predictions = metrics.compute_predictions(heatmaps, pafs, metadata)
        utils.update_predictions_file(cur_predictions, test_exp)

Processing image 0
Processing image 5
Processing image 10
Processing image 15


In [8]:
data_path = CONFIG["paths"]["data_path"]
labels_path = os.path.join(data_path, "annotations")
labels_file = os.path.join(labels_path, "person_keypoints_validation.json")
preds_file = os.path.join(test_exp, CONFIG["paths"]["submission"])

In [9]:
stats = metrics.compute_precision(labels_file=labels_file, preds_file=preds_file)

loading annotations into memory...
Done (t=0.86s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *keypoints*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.305
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.196
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.189
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.184
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.229
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.333
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.250
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | ma

In [10]:
# stats